# A Rod-Pendulum System

The following system will be simulated:

<p align="center">
  <img src="rod-pendulum.png" width="400" /><br>
  taken from <em>Dr. Thomas Erben, Argelander-Institut für Astronomie</em>
</p>

The massless rod of length $2L$ can rotate freely around the origin and is assumed to be massless. The system has two mounts (point masses $m_1$ and $m_2$) at the ends of the rod to which two pendulums are connected. The pendulums (lengths $l_1$ and $l_2$) can rotate freely around the mounts. The pendulums consist of two point masses $m_3$ and $m_4$ but are massless otherwise. The only external force on the system is gravitation and the systems movement all take place in the two-dimensional $x-y$ plane.\
To simplify things, all masses are assumed equal, i.e. $m = m_1 = m_2 = m_3 = m_4$ and also $l = l_1 = l_2$ with $l < L$.
    
The entire notebook should run in about 1 min.

## Part 1
- Finding generalised coordinates

## Part 2
- Construction of the Lagrangian and the equations of motion using SymPy.

## Part 3
- Solving the equations of motion numerically using `odeint` and `solve_ivp`.
- Four different initial conditions leading to linear as well as chaotic behavior.
- Plot of the generalized coordinates over the course of 50 seconds.

## Part 4
- Plot of the time evolution of the system's total energy.
- Comparison of both solvers.

## Part 5
- Plot of the time evolution of the energies from rod (masses $m_1$, $m_2$) and the pendulums (masses $m_3$, $m_4$). 
- Comparison of both solvers.

## Part 6
- Creation of an animated gif showing the systems movement.

---

## Part 1

The entire system can be characterised by three angles. Choosing every angle to the right of a vertical line pointing down, I denote $\theta_1$ as the angle of the rod's right arm as well as $\theta_2$ and $\theta_3$ as the angle of the right and left pendulum, respectively.\
This results in
\begin{align}
x_1 = L \sin \theta_1 \:&,\quad y_1 = -L \cos \theta_1 \\
x_2 = -x_1 \:&,\quad y_2 = -y_1 \\
x_3 = x_1 + l \sin \theta_2 \:&,\quad y_3 = y_1 - l \cos \theta_2 \\
x_4 = -x_1 + l \sin \theta_3 \:&,\quad y_4 = -y_1 - l \cos \theta_3 \,.
\end{align}

## Part 2

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import sympy as sp  # to deal with symbolic expressions
import scipy.integrate as si  # to solve ODEs
from matplotlib import animation  # to animate the system's movement

plt.rcParams.update({'font.size': 14})  # larger text in plots

In [ ]:
# define sympy quantities
L, l, m, g, t = sp.symbols(r'L l m g t')  # constants
theta1, theta2, theta3 = sp.symbols(r'\theta_1 \theta_2 \theta_3', cls=sp.Function)
theta1, theta2, theta3 = theta1(t), theta2(t), theta3(t)  # time-dependent

# define time-derivatives for better readability later
theta1_d, theta2_d, theta3_d = sp.diff(theta1, t), sp.diff(theta2, t), sp.diff(theta3, t)
theta1_dd, theta2_dd, theta3_dd = sp.diff(theta1_d, t), sp.diff(theta2_d, t), sp.diff(theta3_d, t)

# cartesian coordinates (cf. part 1). Tracking m_2 is unnecessary, as (x_2, y_2) = -(x_1, y_1)
x1, y1 = L * sp.sin(theta1), -L * sp.cos(theta1)
x3, y3 = x1 + l * sp.sin(theta2), y1 - l * sp.cos(theta2)
x4, y4 = -x1 + l * sp.sin(theta3), -y1 - l * sp.cos(theta3)

# calculate kinetic terms, potential energy and the Lagrangian

# function to get T from x(t) and y(t)
T = lambda x, y: sp.Rational(1, 2) * m * (sp.diff(x, t)**2 + sp.diff(y, t)**2)

V = m * g * (y3 + y4)  # y1-/y2-contributions cancel

Lagrange = 2 * T(x1, y1) + T(x3, y3) + T(x4, y4) - V  # same m1-/m2-contributions => factor of 2
Lagrange = Lagrange.simplify()

# function to get Euler-Lagrange equations for theta(t) and theta_d(t)
EL_eq = lambda theta, theta_d: sp.diff(Lagrange, theta) - sp.diff(sp.diff(Lagrange, theta_d), t)

LE1, LE2, LE3 = EL_eq(theta1, theta1_d), EL_eq(theta2, theta2_d), EL_eq(theta3, theta3_d)

# convert Euler-Lagrange equations to desired form, i.e. theta_dd = ...
sol = sp.solve([LE1.simplify(), LE2.simplify(), LE3.simplify()], [theta1_dd, theta2_dd, theta3_dd])

# convert symbolic expressions to numerical functions
quantities = [theta1, theta2, theta3, theta1_d, theta2_d, theta3_d, L, l, m, g]

theta1_dd_n = sp.lambdify(quantities, sol[theta1_dd].simplify())
theta2_dd_n = sp.lambdify(quantities, sol[theta2_dd].simplify())
theta3_dd_n = sp.lambdify(quantities, sol[theta3_dd].simplify())

In [ ]:
# to show the results a print()-statement would only output latex code
# so each expression is presented with individual cells 
Lagrange

In [ ]:
LE1  # first Euler-Lagrange equation

In [ ]:
LE2  # second Euler-Lagrange equation

In [ ]:
LE3  # third Euler-Lagrange equation

## Part 3

Since `odeint` can only deal with first order differential equations the three second order differential equations from the previous task need to be converted, such that one gets a system containing six equations. This must be expressed as
\begin{align}
\frac{d\vec{S}}{dt} =
\begin{bmatrix} \dot{\theta_1}\\ \dot{\theta_2} \\ \dot{\theta_3} \\ \dot{u_1} \\ \dot{u_2} \\ \dot{u_3} \end{bmatrix} =
\begin{bmatrix} u_1\\ u_2 \\ u_3 \\ \texttt{LE1} \\ \texttt{LE2} \\ \texttt{LE3} \end{bmatrix}
\end{align}
where $\texttt{LE1}$, $\texttt{LE2}$ and $\texttt{LE3}$ are taken from part 2.

In [ ]:
def dSdt(S, t, L, l, m, g):
    '''
    take a list of quantities, S, representing a vector of angles and their derivatives,
    implement dS/dt based on the lengths (L, l) and masses (m) of the system 
    and the gravitational constant (g)
    return dS/dt as a list    
    '''
    
    theta1, theta2, theta3, u1, u2, u3 = S
    
    # get dS/dt entrywise as explained above, using the numerical functions obtained with sympy
    dtheta1dt, dtheta2dt, dtheta3dt = u1, u2, u3
    du1dt = theta1_dd_n(theta1, theta2, theta3, u1, u2, u3, L, l, m, g)
    du2dt = theta2_dd_n(theta1, theta2, theta3, u1, u2, u3, L, l, m, g)
    du3dt = theta3_dd_n(theta1, theta2, theta3, u1, u2, u3, L, l, m, g)
    
    return [dtheta1dt, dtheta2dt, dtheta3dt, du1dt, du2dt, du3dt]


def get_cartesian(theta1: float, theta2: float, theta3: float,
                  L: float, l: float) -> np.ndarray:
    '''
    take the three generalised coordinates from task 1 and the relevant lengths
    return cartesian coordinates of the three masses
    '''
    
    x1, y1 = L * np.sin(theta1), -L * np.cos(theta1)
    x2, y2 = -x1, -y1
    x3, y3 = x1 + l * np.sin(theta2), y1 - l * np.cos(theta2)
    x4, y4 = -x1 + l * np.sin(theta3), -y1 - l * np.cos(theta3)
    
    return np.array((x1, y1, x2, y2, x3, y3, x4, y4))


def solve_ode(L: float, l: float, m: float, g: float,
              t_n: np.ndarray, S0: list, cartesian: bool, ivp=False) -> np.ndarray:
    '''
    @param L: rod's lenght
    @param l: pendula's lengths
    @param m: masses of the objects
    @param g: gravitational constant
    @param t_n: time steps to solve the ODEs
    @param S0: initial conditions
    @param cartesian: return cartesian coordinates? (otherwise angles)
    @param ivp: solve with 'solve_ivp'? (otherwise 'odeint')
    
    solve the system of ODEs
    return solution in desired coordinates at given time steps
    '''
    
    if ivp:
        # define wrapper because solve_ivp needs f(t, S) (mind the order)
        fun = lambda t, S: dSdt(S, t, L, l, m, g)
        
        # solve numerically and store only y-part of the output, i.e. coordinates
        sol = si.solve_ivp(fun, t_span=(t_n[0], t_n[-1]), y0=S0, t_eval=t_n)
        sol_dgl = np.transpose(sol.y)  # transpose to match odeint's ouput shape: (len(t), len(S0))
        
    else:
        # solve with 'odeint' as in the lecture
        sol_dgl = si.odeint(dSdt, S0, t=t_n, args=(L, l, m, g))
    
    if cartesian == False:
        return np.array((sol_dgl[:,0], sol_dgl[:,1], sol_dgl[:,2]))
    
    else:
        return get_cartesian(sol_dgl[:,0], sol_dgl[:,1], sol_dgl[:,2], L, l)


# solve ODEs for 4 testcases. Using both 'odeint' and 'solve_ivp'
# use identical time steps for each simulation
n_seconds = 50
n_frames = 1500
t_n = np.linspace(0.0, n_seconds, n_frames)

# case 1: linear evolution (lightly move m1 and displace m3)
S0_1 = [np.pi / 2., np.pi / 100., 0, -np.pi / 500., 0, 0]  # order: angles, angular velocities

# case 2: linear evolution (move m3 and m4 oppositely)
S0_2 = [np.pi / 2., 0, 0, 0, np.pi / 4., -np.pi / 4.]

# case 3: chaotic evolution (move m1 and m3)
S0_3 = [np.pi / 2., 0, 0, np.pi, np.pi, 0]

# case 4: chaotic evolution (displace m3)
S0_4 = [np.pi / 2., np.pi / 2., 0, 0, 0, 0]

# define lists to loop over while generating the data to plot
S0_tot = [S0_1, S0_1, S0_2, S0_2, S0_3, S0_3, S0_4, S0_4]
ivp_usage = [False, True] * 4

# creat plot
fig, ax = plt.subplots(4, 2, figsize=(10,14))
fig.tight_layout(pad=2.5)

for ind, axes in enumerate(ax.flat):
    # generate data
    thetas = solve_ode(1.0, 0.5, 1.0, 9.81, t_n, S0_tot[ind],
                       cartesian=False, ivp=ivp_usage[ind])
    
    # plot angles in degrees for better intuition, do not use '... mod 360' due to visability
    axes.plot(t_n, np.rad2deg(thetas[0]), label=r'$\theta_1$')
    axes.plot(t_n, np.rad2deg(thetas[1]), label=r'$\theta_2$')
    axes.plot(t_n, np.rad2deg(thetas[2]), label=r'$\theta_3$')

    axes.set_xlabel(r'$t$ in s')
    axes.set_ylabel(r'$\theta$ in °')
    axes.legend()
    axes.grid(True, 'both')

ax[0,0].set_title(r'$\mathtt{odeint}$', y=1.1)  # y-argument to manually set height
ax[0,1].set_title(r'$\mathtt{solve\_ivp}$', y=1.1)
fig.suptitle('Rod-pendulum system for different initial conditions', y=1.03)

plt.show()

### Comparison of both solvers

In the first two cases, where the system behaves almost linearly, both solvers yield very identical solutions.\
However in the chaotic cases the solutions seemingly differ from each other.
In the last case this can be seen after around $30$s where the envelopes of the oscillations start to differ, while the overall movement is still similar. For the third case the difference of both solvers is very notable.
This can be already seen after around $5$s after which both solvers yield completely different solutions.

## Part 4

In [ ]:
def get_energies(L: float, l: float, m: float, g: float,
                 t_n: np.ndarray, S0: list, ivp=False) -> dict:
    '''
    @param L: rod's lenght
    @param l: pendula's lengths
    @param m: masses of the objects
    @param g: gravitational constant
    @param t_n: time steps to solve the ODEs
    @param S0: initial conditions
    @param ivp: solve with 'solve_ivp'? (otherwise 'odeint')
    
    use previously defined solve_ode() to solve ODEs for a given case
    calculate the rod's and pendula's energies
    return total-/ pendula- and rod-energies for each time step
    '''
    
    positions = solve_ode(L, l, m, g, t_n, S0, cartesian=True, ivp=ivp)
    
    # numerically differentiate positions to get velocities (reduces length by one)
    dt = t_n[1] - t_n[0]
    velocities = (positions[:,1:] - positions[:,:-1]) / dt
    
    # sum squared velocities for constant-t-steps
    T_rod = .5 * m * np.sum(velocities[:4]**2, axis=0)
    T_pendula = .5 * m * np.sum(velocities[4:]**2, axis=0)
    
    # get potential energy (vanishes for the rod)
    V_pendula = m * g * (positions[5] + positions[7])
    
    # get total pendula-energy. Drop V[-1], s.t. T and V have the same length
    E_pendula = T_pendula + V_pendula[:-1]
    
    return {'tot': E_pendula + T_rod, 'pendula': E_pendula, 'rod': T_rod}
    

# generate results
# take longer times and consider two different step sizes in time
n_seconds = 200
dt_fine = 0.01
dt_coarse = 0.1
t_data = [np.arange(0.0, n_seconds, dt_fine), np.arange(0.0, n_seconds, dt_coarse)]

# consider similar cases as in task 3,
# only slighty change case 1 (increase l, s.t. energies differ from case 2)
# specify simulations with lists structured as: [S0, l, use ivp?, save data for task5?]
specifications = [[S0_1, 0.9, False, False], [S0_2, 0.5, False, True],
                  [S0_3, 0.5, False, True], [S0_4, 0.5, False, False],       
                  [S0_1, 0.9, True, False], [S0_2, 0.5, True, True],
                  [S0_3, 0.5, True, True], [S0_4, 0.5, True, False]]
specifications *= 2  # duplicate to consider two step sizes in time

# define containers to store data
max_variations = np.zeros(16)

# store data for task 5 to not run the simulations again
data_rod = [None] * 8
data_pendula = [None] * 8
i = 0  # counter variable, will be used when filling data_rod/data_pendula


# prepare plot for the following loops
fig, ax = plt.subplots(2, 2, figsize=(10,8))
fig.tight_layout(pad=3.5)

# loop over previously defined configurations to generate and plot data
for ind, specify in enumerate(specifications):
    # unpack for better readability
    S0, length, use_ivp, save_for_task5 = specify
    
    # index t_data with 'ind // 8' to adress each entry 8 times. Avoids storing t-arrays several times
    energies = get_energies(1., length, 1., 9.81, t_data[ind // 8], S0, use_ivp)
    
    # quantify maximal changes in energy over time
    max_variations[ind] = np.max(energies['tot']) - np.min(energies['tot'])
    
    # safe data now to not do the same simulations in task 5
    if save_for_task5 == True:
        data_rod[i] = energies['rod']
        data_pendula[i] = energies['pendula']
        i += 1
        
    # plot total energies
    label = str(ind % 4 + 1)  # number the cases for the legend
    # index with 'ind //...' to adress entries multiple times. Drop last time step to match energy's length
    ax.flat[ind // 4].plot(t_data[ind // 8][:-1], energies['tot'], label=label)

    
# set up the rest of the plot
subtitles = [r'$\mathtt{odeint}$ for $\Delta t=0.01$', r'$\mathtt{ivp}$ for $\Delta t=0.01$',
             r'$\mathtt{odeint}$ for $\Delta t=0.1$', r'$\mathtt{ivp}$ for $\Delta t=0.1$']

for axes, title in zip(ax.flat, subtitles):
    axes.set_title(title)
    axes.set_xlabel(r'$t$ in s')
    axes.set_ylabel(r'$E$ in J')
    axes.legend()
    axes.grid(True, 'both')

fig.suptitle("Time evolution of the rod-pendulum system's energy for 4 different cases", y=1.02)

plt.show()


# prepare output of maximal variations with pandas (automatically generates tables)
# reshape max_variations, s.t. indices represent: fine/coarse, odeint/solve_ivp, case
max_variations = max_variations.reshape(2, 2, 4)

# split up variations in fine/coarse to generate two tables
variations_fine = np.transpose(max_variations[0])  # transpose for similar structure as in the plot
variations_coarse = np.transpose(max_variations[1])

# define pandas data frames
df_fine = pd.DataFrame(variations_fine, columns=['odeint', 'solve_ivp'],
                       index=['case 1', 'case 2', 'case 3', 'case 4'])
df_coarse = pd.DataFrame(variations_coarse, columns=['odeint', 'solve_ivp'],
                         index=['case 1', 'case 2', 'case 3', 'case 4'])

print('The maximal variations (in Joule) of the respective simulations are:')

# print data frames as tables
print(f'\n For time steps of {dt_fine}:')
print(df_fine)
print(f'\n For time steps of {dt_coarse}:')
print(df_coarse)

### Discussion

The expectation is of course that the total energy should be conserved, i.e. constant over time.\
However, for the numerical solutions there are deviations from the energy conservation depending on the considered case, the solver and the time steps:

- The case-dependence seems to be determined by how chaotic the system behaves. The more chaotic the systems evolves, the more changes the energy over time. Thus the cases 1 and 2 seem to always fullfill the expectation quite accurately (the variation remains far under 1 J for all simulations) while especially case 3 yields high changes in energy.

- At small time scales both solvers give energies that seem to oscillate around a certain value with similar amplitudes. This can be also seen considering the variations where `odeint` is only slightly more stable than `solve_ivp`. However, there are differences at longer scales if the system is very chaotic. Case 3 reflects this as the oscillations' envelope stays relatively constant for `odeint` while it appears to grow exponentially for `solve_ivp`, yielding maximal variations that are larger by factors of about $182$ and $14$ depending on the time steps. The reason is that both solvers use different numerical methods to solve the differential equations, namely Adams/BDF for `odeint` and Runge-Kutta for `solve_ivp`. Those methods accumulate numerical errors differently.

- The main influence of the time steps seems to be that larger steps lead to an increase of the oscillations' amplitudes, where the variations seem to scale linearly with the step size in most simulations (cases 1 and 2 for `solve_ivp` and all cases for `odeint`). In addition, solving case 3 with `solve_ivp` one can observe that the envelope grows more slowly for larger time steps. Leading to a decrease in the maximal variation.

## Part 5

In [ ]:
# consider cases 2 and 3, i.e. linear and chaotic behavior
# use the same time steps as in part 4, s.t. data can be reused

# subtitles for the plot
subtitles = [r'case 2: $\mathtt{odeint}$ for $\Delta t=0.01$',
             r'case 3: $\mathtt{odeint}$ for $\Delta t=0.01$',
             r'case 2: $\mathtt{solve\_ivp}$ for $\Delta t=0.01$',
             r'case 3: $\mathtt{solve\_ivp}$ for $\Delta t=0.01$',
             r'case 2: $\mathtt{odeint}$ for $\Delta t=0.1$',
             r'case 3: $\mathtt{odeint}$ for $\Delta t=0.1$',
             r'case 2: $\mathtt{solve\_ivp}$ for $\Delta t=0.1$',
             r'case 3: $\mathtt{solve\_ivp}$ for $\Delta t=0.1$']

# plot the results
fig, ax = plt.subplots(4, 2, figsize=(10,14))
fig.tight_layout(pad=3.5)

# reorder subplots for consistent plot structure
ax[:2,:] = np.transpose(ax[:2,:])
ax[2:,:] = np.transpose(ax[2:,:])

for ind, axes in enumerate(ax.flat):
    # do not plot errorbars as nothing would be visible
    axes.plot(t_data[ind // 4][:-1], data_rod[ind], label='Rod')
    axes.plot(t_data[ind // 4][:-1], data_pendula[ind], label='Pendula')

    axes.set_title(subtitles[ind])
    axes.set_xlabel(r'$t$ in s')
    axes.set_ylabel(r'$E$ in J')
    axes.legend()
    axes.grid(True, 'both')

fig.suptitle("Seperate time evolution of the rod- and pendula-energy", y=1.02)

plt.show()

### Discussion

Since the itself can only have kinetic energy one would expect its energy to be bound from below at 0 J. Due to energy conservation the pendula's energy should be bound from above at the total energy.
For simulations with small time steps those expectations seem to be fullfilled well. The pendula's energy seems to be bounded by a course that looks similar to the envelop of the respective simulation in task 4.
Only the solution of case 3 with `odeint` on the coarse time grid seem to violate the expectation because one cannot observe a clear upper bound for the energy of the pendula.\
Another observation is that the energy is translated much more between the rod and the pendula in the chaotic case. Meanwhile, in the more linear 2nd case both energies seem to remain constant. This makes sense because case 2 was constructed in a way that the rod does not move at all.

## Part 6

In [ ]:
# define test case
n_seconds = 10
n_frames = 300  # 30 frames per second
t_animate = np.linspace(0.0, n_seconds, n_frames)

# consider slightly displace from position of maximal potential
S0_animate = [0, np.pi, np.pi * (1 + .01), 0, 0, 0]

# solve numerically and extract coordinates
positions = solve_ode(1., 0.8, 1., 9.81, t_animate, S0_animate, cartesian=True, ivp=False)
x1_n, y1_n, x2_n, y2_n, x3_n, y3_n, x4_n, y4_n = tuple(positions)


# define animation function (cannot be done before since it needs coordinates)
def animate(i, ln1):
    # line connecting m4, m2, m1 and m3
    ln1.set_data([x4_n[i], x2_n[i], x1_n[i], x3_n[i]],
                 [y4_n[i], y2_n[i], y1_n[i], y3_n[i]])
    
    # second line connecting m1 and m2 to draw the rod thicker
    ln2.set_data([x2_n[i], x1_n[i]],[y2_n[i], y1_n[i]])
    
    # mark coordinates of the massess for individual color
    m1.set_data([x1_n[i]], [y1_n[i]])
    m2.set_data([x2_n[i]], [y2_n[i]])
    m3.set_data([x3_n[i]], [y3_n[i]])
    m4.set_data([x4_n[i]], [y4_n[i]])
    
    return ln1,


# plot the system based on the given sketch
fig, ax = plt.subplots(1,1, figsize=(8,8))

# represent mounting
plt.vlines(0., -2., 0., colors='gray', lw=12)
plt.hlines(-2., -1., 1., colors='gray', lw=12)

# represent rod and pendula with different thickness
ln1, = plt.plot([], [], lw=3, color='black')
ln2, = plt.plot([], [], lw=7, color='black')

# represent masses in different colors
m1, = plt.plot([], [], color='blue', markersize=12, marker='o')
m2, = plt.plot([], [], color='red', markersize=12, marker='o')
m3, = plt.plot([], [], color='orange', markersize=12, marker='o')
m4, = plt.plot([], [], color='green', markersize=12, marker='o')

# represent attachment of the rod on the mounting
plt.plot(0, 0, markersize=18, marker='o', color='black', markerfacecolor='black')

ax.set_ylim(-2, 2)
ax.set_xlim(-2, 2)
ax.grid()
ax.set_aspect('equal')  # identical scaling of x and y

# create and safe animation as shown in the lecture
ani = animation.FuncAnimation(fig, animate, fargs=(ln1,), frames=n_frames)
ani.save('pendulum.gif', writer='pillow', fps=(n_frames / n_seconds))